In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotnine import *


In [2]:
# Read data for each year

df23 = pd.read_csv('d23.csv')
df23.rename(columns={'PAGU DIPA AWAL KL': 'PAGU DIPA AWAL'}, inplace=True)
df23.rename(columns={'PAGU DIPA REVISI KL': 'PAGU DIPA REVISI'}, inplace=True)
df23.rename(columns={'PAGU HIMPUNAN KL': 'PAGU HIMPUNAN'}, inplace=True)
df23.rename(columns={'BLOKIR DIPA AWAL KL': 'BLOKIR DIPA AWAL'}, inplace=True)
df23.rename(columns={'BLOKIR DIPA REVISI KL': 'BLOKIR DIPA REVISI'}, inplace=True)
df23.rename(columns={'BLOKIR HIMPUNAN KL': 'BLOKIR HIMPUNAN'}, inplace=True)
df23['Tahun'] = '2023'

df24 = pd.read_csv('d24.csv')
df24.rename(columns={'PAGU DIPA AWAL KL': 'PAGU DIPA AWAL'}, inplace=True)
df24.rename(columns={'PAGU DIPA REVISI KL': 'PAGU DIPA REVISI'}, inplace=True)
df24.rename(columns={'PAGU KEPPRES': 'PAGU PERPRES'}, inplace=True)
df24.rename(columns={'BLOKIR DIPA AWAL KL': 'BLOKIR DIPA AWAL'}, inplace=True)
df24.rename(columns={'BLOKIR DIPA REVISI KL': 'BLOKIR DIPA REVISI'}, inplace=True)
df24.rename(columns={'BLOKIR KEPPRES': 'BLOKIR PERPRES'}, inplace=True)
df24['Tahun'] = '2024'

df25 = pd.read_csv('d25.csv')
df25.rename(columns={'PAGU PERPRES MP': 'PAGU PERPRES'}, inplace=True)
df25.rename(columns={'BLOKIR PERPRES MP': 'BLOKIR PERPRES'}, inplace=True)
df25['Tahun'] = '2025'

df26 = pd.read_csv('d26.csv')
df26['Tahun'] = '2026'


In [3]:
# Create a dictionary to hold column names for each dataframe
column_dict = {
    'df23': df23.columns,
    'df24': df24.columns,
    'df25': df25.columns,
    'df26': df26.columns
}

# Get a set of all unique column names across all dataframes
all_columns = set()
for cols in column_dict.values():
    all_columns.update(cols)

# Create a comparison dataframe
comparison_df = pd.DataFrame(index=sorted(list(all_columns)))

# Populate the comparison dataframe
for df_name, cols in column_dict.items():
    comparison_df[df_name] = comparison_df.index.isin(cols)

# Display the comparison table
print("Column presence in each DataFrame (True = exists, False = does not exist):")
print(comparison_df)

Column presence in each DataFrame (True = exists, False = does not exist):
                               df23  df24  df25   df26
AKUN 4 DIGIT                   True  True  True   True
BLOKIR DIPA AWAL               True  True  True   True
BLOKIR DIPA REVISI             True  True  True  False
BLOKIR HIMPUNAN                True  True  True   True
BLOKIR PERPRES                False  True  True   True
FUNGSI                         True  True  True   True
JENIS BELANJA                  True  True  True   True
KEGIATAN                       True  True  True   True
KEMENTERIAN/LEMBAGA            True  True  True   True
KOMPONEN                       True  True  True   True
OUTPUT (KRO)                   True  True  True   True
PAGU DIPA AWAL                 True  True  True   True
PAGU DIPA REVISI               True  True  True  False
PAGU HIMPUNAN                  True  True  True   True
PAGU PERPRES                  False  True  True   True
PROGRAM                        True  True  Tr

In [4]:
df = pd.concat([df23, df24, df25, df26], ignore_index=True).reset_index(drop=True)
df['PAGU DIPA AWAL EFEKTIF'] = df['PAGU DIPA AWAL'] - df['BLOKIR DIPA AWAL']
df['PAGU DIPA REVISI EFEKTIF'] = df['PAGU DIPA REVISI'] - df['BLOKIR DIPA REVISI']
df['PAGU HIMPUNAN EFEKTIF'] = df['PAGU HIMPUNAN'] - df['BLOKIR HIMPUNAN']
df['PAGU PERPRES EFEKTIF'] = df['PAGU PERPRES'] - df['BLOKIR PERPRES']

In [5]:
# Ensure Tahun is string and trimmed (works whether it's numeric or string)
df['Tahun'] = df['Tahun'].astype(str).str.strip()

# Create a helper column with the leading token (the "digit" before the first space)
df['kl_digit'] = df['KEMENTERIAN/LEMBAGA'].astype(str).str.strip().str.split().str[0]

# Build mapping: digit -> full KEMENTERIAN/LEMBAGA from Tahun == 2025
# If multiple 2025 rows share the same digit we take the most common name (mode) or first one
mapping_series = (
    df[df['Tahun'] == '2025']
    .groupby('kl_digit')['KEMENTERIAN/LEMBAGA']
    .agg(lambda x: x.mode().iat[0] if not x.mode().empty else x.iloc[0])
)

mapping = mapping_series.to_dict()

if not mapping:
    st.warning("Tidak ada mapping dari Tahun 2025 ditemukan (kolom 'KEMENTERIAN/LEMBAGA' mungkin kosong).")
else:
    # Apply mapping to rows in 2023 and 2024 where kl_digit exists in mapping
    mask = df['Tahun'].isin(['2023', '2024']) & df['kl_digit'].isin(mapping.keys())
    # Map kl_digit -> full 2025 name
    df.loc[mask, 'KEMENTERIAN/LEMBAGA'] = df.loc[mask, 'kl_digit'].map(mapping)

# Clean up helper column
df = df.drop(columns=['kl_digit'])

df['PROGRAM'] = df['PROGRAM'].str[4:]

# Remove 4-digit prefix from KOMPONEN when the first 4 chars are digits

mask = df['KOMPONEN'].notna() & df['KOMPONEN'].str[:4].str.isdigit()
df.loc[mask, 'KOMPONEN'] = df.loc[mask, 'KOMPONEN'].str[4:].str.lstrip()

# Drop sentinel/placeholder K/L rows (case-insensitive, handles variants starting with '999')
df['KEMENTERIAN/LEMBAGA'] = df['KEMENTERIAN/LEMBAGA'].astype(str).str.strip()
drop_mask = df['KEMENTERIAN/LEMBAGA'].str.upper().str.contains(r'(^999\b)|BAGIAN ANGGARAN BENDAHARA UMUM NEGARA', na=False)
n_dropped = drop_mask.sum()
if n_dropped:
    print(f"Dropping {n_dropped} rows matching placeholder K/L")
df = df[~drop_mask].reset_index(drop=True)



C:\Users\USER\AppData\Local\Temp\ipykernel_3232\1649442922.py:37: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.


Dropping 233 rows matching placeholder K/L


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660403 entries, 0 to 660402
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   KEMENTERIAN/LEMBAGA           660403 non-null  object 
 1   SUMBER DANA                   660403 non-null  object 
 2   FUNGSI                        660403 non-null  object 
 3   SUB FUNGSI                    660403 non-null  object 
 4   PROGRAM                       660403 non-null  object 
 5   KEGIATAN                      660403 non-null  object 
 6   OUTPUT (KRO)                  660403 non-null  object 
 7   SUB OUTPUT (RO)               660403 non-null  object 
 8   KOMPONEN                      660403 non-null  object 
 9   JENIS BELANJA                 660403 non-null  object 
 10  AKUN 4 DIGIT                  660403 non-null  object 
 11  REALISASI BELANJA KL (SAKTI)  323742 non-null  float64
 12  PAGU DIPA REVISI              352858 non-nul

In [7]:
df.columns

Index(['KEMENTERIAN/LEMBAGA', 'SUMBER DANA', 'FUNGSI', 'SUB FUNGSI', 'PROGRAM',
       'KEGIATAN', 'OUTPUT (KRO)', 'SUB OUTPUT (RO)', 'KOMPONEN',
       'JENIS BELANJA', 'AKUN 4 DIGIT', 'REALISASI BELANJA KL (SAKTI)',
       'PAGU DIPA REVISI', 'BLOKIR DIPA REVISI', 'PAGU DIPA AWAL',
       'BLOKIR DIPA AWAL', 'PAGU HIMPUNAN', 'BLOKIR HIMPUNAN', 'Tahun',
       'PAGU PERPRES', 'BLOKIR PERPRES', 'PAGU DIPA AWAL EFEKTIF',
       'PAGU DIPA REVISI EFEKTIF', 'PAGU HIMPUNAN EFEKTIF',
       'PAGU PERPRES EFEKTIF'],
      dtype='object')

In [8]:
df.to_csv('df.csv')

## Cleaning for Monitoring

In [ ]:
df_04 = pd.read_csv('df_20251204.csv')
df_04['Tahun'] = '2026'
df_04['PAGU DIPA AWAL EFEKTIF'] = df_04['PAGU DIPA AWAL'] - df_04['BLOKIR DIPA AWAL']
df_04['PAGU HIMPUNAN EFEKTIF'] = df_04['PAGU HIMPUNAN'] - df_04['BLOKIR HIMPUNAN']
df_04['PAGU PERPRES EFEKTIF'] = df_04['PAGU PERPRES'] - df_04['BLOKIR PERPRES']
df_04.to_csv('df_20251204_cleaned.csv')


In [15]:
df_04

,KEMENTERIAN/LEMBAGA,UNIT/ESELON I,SATKER,SUMBER DANA,FUNGSI,SUB FUNGSI,PROGRAM,KEGIATAN,OUTPUT (KRO),SUB OUTPUT (RO),...,PAGU DIPA AWAL,BLOKIR DIPA AWAL,PAGU PERPRES,BLOKIR PERPRES,PAGU HIMPUNAN,BLOKIR HIMPUNAN,Tahun,PAGU DIPA AWAL EFEKTIF,PAGU HIMPUNAN EFEKTIF,PAGU PERPRES EFEKTIF
0,001 MAJELIS PERMUSYAWARATAN RAKYAT (MPR),001 01 Deputi Bidang Administrasi,000017 SEKRETARIAT JENDERAL,01 RUPIAH MURNI,01 PELAYANAN UMUM,"01 01 LEMBAGA EKSEKUTIF DAN LEGISLATIF, MASALA...",001 01 WA Program Dukungan Manajemen,1001 Pengelolaan Administrasi dan Fasilitasi R...,1001 EBA Layanan Dukungan Manajemen Internal,1001 EBA 957 Layanan Hukum,...,NaN,NaN,143400000.0,0.0,145020000.0,0.0,2026,NaN,145020000.0,143400000.0
1,001 MAJELIS PERMUSYAWARATAN RAKYAT (MPR),001 01 Deputi Bidang Administrasi,000017 SEKRETARIAT JENDERAL,01 RUPIAH MURNI,01 PELAYANAN UMUM,"01 01 LEMBAGA EKSEKUTIF DAN LEGISLATIF, MASALA...",001 01 WA Program Dukungan Manajemen,1001 Pengelolaan Administrasi dan Fasilitasi R...,1001 EBA Layanan Dukungan Manajemen Internal,1001 EBA 957 Layanan Hukum,...,NaN,NaN,14400000.0,0.0,14400000.0,0.0,2026,NaN,14400000.0,14400000.0
2,001 MAJELIS PERMUSYAWARATAN RAKYAT (MPR),001 01 Deputi Bidang Administrasi,000017 SEKRETARIAT JENDERAL,01 RUPIAH MURNI,01 PELAYANAN UMUM,"01 01 LEMBAGA EKSEKUTIF DAN LEGISLATIF, MASALA...",001 01 WA Program Dukungan Manajemen,1001 Pengelolaan Administrasi dan Fasilitasi R...,1001 EBA Layanan Dukungan Manajemen Internal,1001 EBA 957 Layanan Hukum,...,NaN,NaN,8160000.0,0.0,8160000.0,0.0,2026,NaN,8160000.0,8160000.0
3,001 MAJELIS PERMUSYAWARATAN RAKYAT (MPR),001 01 Deputi Bidang Administrasi,000017 SEKRETARIAT JENDERAL,01 RUPIAH MURNI,01 PELAYANAN UMUM,"01 01 LEMBAGA EKSEKUTIF DAN LEGISLATIF, MASALA...",001 01 WA Program Dukungan Manajemen,1001 Pengelolaan Administrasi dan Fasilitasi R...,1001 EBA Layanan Dukungan Manajemen Internal,1001 EBA 957 Layanan Hukum,...,NaN,NaN,37152000.0,0.0,36180000.0,0.0,2026,NaN,36180000.0,37152000.0
4,001 MAJELIS PERMUSYAWARATAN RAKYAT (MPR),001 01 Deputi Bidang Administrasi,000017 SEKRETARIAT JENDERAL,01 RUPIAH MURNI,01 PELAYANAN UMUM,"01 01 LEMBAGA EKSEKUTIF DAN LEGISLATIF, MASALA...",001 01 WA Program Dukungan Manajemen,1001 Pengelolaan Administrasi dan Fasilitasi R...,1001 EBA Layanan Dukungan Manajemen Internal,1001 EBA 957 Layanan Hukum,...,NaN,NaN,10800000.0,0.0,10800000.0,0.0,2026,NaN,10800000.0,10800000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,025 KEMENTERIAN AGAMA,025 04 Ditjen Pendidikan Islam,575422 MADRASAH ALIYAH NEGERI 5 GARUT KAB. GARUT,01 RUPIAH MURNI,10 PENDIDIKAN,10 02 PENDIDIKAN DASAR DAN MENENGAH,025 04 DF Program Wajib Belajar 13 Tahun,2129 Pengelolaan dan Pembinaan Pendidikan Madr...,2129 QEN Tidak Terdapat di Referensi,2129 QEN 007 Siswa MA Penerima BOS,...,85008000.0,0.0,85008000.0,0.0,NaN,NaN,2026,85008000.0,NaN,85008000.0
499996,025 KEMENTERIAN AGAMA,025 04 Ditjen Pendidikan Islam,575422 MADRASAH ALIYAH NEGERI 5 GARUT KAB. GARUT,01 RUPIAH MURNI,10 PENDIDIKAN,10 90 PENDIDIKAN DAN KEBUDAYAAN LAINNYA,025 04 WA Program Dukungan Manajemen,2135 Dukungan Manajemen Pendidikan dan Pelayan...,2135 EBA Layanan Dukungan Manajemen Internal,2135 EBA 994 Layanan Perkantoran,...,NaN,NaN,84039000.0,0.0,84039000.0,0.0,2026,NaN,84039000.0,84039000.0
499997,025 KEMENTERIAN AGAMA,025 04 Ditjen Pendidikan Islam,575422 MADRASAH ALIYAH NEGERI 5 GARUT KAB. GARUT,01 RUPIAH MURNI,10 PENDIDIKAN,10 90 PENDIDIKAN DAN KEBUDAYAAN LAINNYA,025 04 WA Program Dukungan Manajemen,2135 Dukungan Manajemen Pendidikan dan Pelayan...,2135 EBA Layanan Dukungan Manajemen Internal,2135 EBA 994 Layanan Perkantoran,...,NaN,NaN,12222000.0,0.0,12222000.0,0.0,2026,NaN,12222000.0,12222000.0
499998,025 KEMENTERIAN AGAMA,025 04 Ditjen Pendidikan Islam,575422 MADRASAH ALIYAH NEGERI 5 GARUT KAB. GARUT,01 RUPIAH MURNI,10 PENDIDIKAN,10 90 PENDIDIKAN DAN KEBUDAYAAN LAINNYA,025 04 WA Program Dukungan Manajemen,2135 Dukungan Manajemen Pendidikan dan Pelayan...,21

In [14]:
df_10 = pd.read_csv('df_20251210.csv')
df_10['Tahun'] = '2026'
df_10['PAGU DIPA AWAL EFEKTIF'] = df_10['PAGU DIPA AWAL'] - df_10['BLOKIR DIPA AWAL']
df_10['PAGU HIMPUNAN EFEKTIF'] = df_10['PAGU HIMPUNAN'] - df_10['BLOKIR HIMPUNAN']
df_10['PAGU PERPRES EFEKTIF'] = df_10['PAGU PERPRES'] - df_10['BLOKIR PERPRES']
df_10.to_csv('df_20251210_cleaned.csv')

In [2]:
df_18 = pd.read_csv('df_20251218.csv')
df_18['Tahun'] = '2026'
df_18['PAGU DIPA AWAL EFEKTIF'] = df_18['PAGU DIPA AWAL'] - df_18['BLOKIR DIPA AWAL']
df_18['PAGU HIMPUNAN EFEKTIF'] = df_18['PAGU HIMPUNAN'] - df_18['BLOKIR HIMPUNAN']
df_18['PAGU PERPRES EFEKTIF'] = df_18['PAGU PERPRES'] - df_18['BLOKIR PERPRES']
df_18.to_csv('df_20251218_cleaned.csv')